<img src="https://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Financial Packages

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

### Please use the "Python 3.10, Numpy 1.26.4 Pandas TA" kernel.

## `eod` package

The package ist install via (see https://pypi.org/project/eod/):

    pip install eod

## Imports 

In [ ]:
!git clone https://github.com/tpq-classes/financial_packages.git
import sys
sys.path.append('financial_packages')


In [ ]:
import requests
import cufflinks as cf
import pandas as pd
from pylab import plt
from io import StringIO
plt.style.use('seaborn-v0_8')
%config InlineBackend.figure_format = 'svg'
cf.set_config_file(offline=True)

# EOD Historical Data

The `eod` package allows the retrieval of historical and real-time financial data from EOD Historical Data. See [company page](https://eodhistoricaldata.com/r/?ref=X8R79ISB). The service is a subscription-based one. There is also a free trial available.

## API Token

The following requires the definition of `api_key` as follows:

    api_key = "YOUR_API_KEY_FROM_EODHD"

In [ ]:
#%run ../../../data/creds.py

## Single Request

Source: ChatGPT 4.0.

In [ ]:
import requests

In [ ]:
def get_financial_data(symbol, api_key):
    base_url = 'https://eodhistoricaldata.com/api'
    endpoint = f'/eod/{symbol}'
    
    params = {
        'api_token': api_key,
        'fmt': 'json',  # Request data in JSON format
    }
    
    url = f'{base_url}{endpoint}'
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f'Error: {response.status_code}')
        return None

In [ ]:
symbol = 'AAPL.US'  # Apple Inc. (AAPL) listed on the US stock exchange

In [ ]:
financial_data = get_financial_data(symbol, api_key)

In [ ]:
print(financial_data[-3:])

## API Connection

In [ ]:
from eod import EodHistoricalData

In [ ]:
api = EodHistoricalData(api_key)

## Exchanges

In [ ]:
all_exchanges = api.get_exchanges()

In [ ]:
ex_df = pd.DataFrame.from_dict(all_exchanges)

In [ ]:
ex_df

In [ ]:
sorted(list(ex_df['Country'].unique()))

## Stock Price Data

* `period='w'` for weekly
* `period='m'` for monthly
* `order='a'` for ascending
* `order='d'` for descending

In [ ]:
symbol = 'AAPL.US'

In [ ]:
%time prices = api.get_prices_eod(symbol, period='d', order='a')

In [ ]:
data = pd.DataFrame.from_dict(prices).set_index('date')

In [ ]:
data.info()  # index is not (yet) a DatetimeIndex

In [ ]:
data.tail()

In [ ]:
data.index = pd.DatetimeIndex(data.index)

In [ ]:
data.info()

In [ ]:
data['adjusted_close'].iloc[-1000:].iplot(title=symbol);

In [ ]:
qf = cf.QuantFig(
         data.iloc[-100:],
         title='Apple Inc.',
         legend='top',
         name='AAPL.O'
)

In [ ]:
qf.add_bollinger_bands()

In [ ]:
qf.iplot()

In [ ]:
sym = ['AAPL.US', 'MSFT.US', 'AMZN.US', 'INTC.US']

In [ ]:
%%time
raw = {}
for s in sym:
    prices = api.get_prices_eod(s, period='d', order='a')
    price_df = pd.DataFrame.from_dict(prices).set_index('date')
    raw[s] = price_df

In [ ]:
data = pd.DataFrame()
for s in raw:
    data[s] = raw[s]['adjusted_close']
data.dropna(inplace=True)

In [ ]:
data.head()

In [ ]:
data.loc['2015-1-1':].normalize(multiplier=1).iplot()

In [ ]:
data.loc['2013-1-1':].normalize(multiplier=1).iplot()

In [ ]:
data.normalize(multiplier=1).iplot();

## ETF Data

Date for the following ETFs is retrieved:

* `IQQW.XETRA`: iShares MSCI World UCITS ETF USD (Dist)
* `QDVE.XETRA`: iShares S&P 500 Info Technolg Sctr UCITS ETF USD A
* `EXXT.XETRA`: iShares NASDAQ-100 UCITS ETF (DE) Closed Fund
* `EGLN.LSE`: iShares Physical Gold ETC

In [ ]:
sym = ['IQQW.XETRA', 'QDVE.XETRA', 'EXXT.XETRA', 'EGLN.LSE']

In [ ]:
%time prices = api.get_prices_eod(sym[1], period='d', order='a')

In [ ]:
price_df = pd.DataFrame.from_dict(prices).set_index('date')

In [ ]:
price_df.info()

In [ ]:
price_df.tail()

In [ ]:
qf = cf.QuantFig(price_df.iloc[-150:],
                title='iShares S&P 500 IT',
                legend='top',
                name=sym[1])

In [ ]:
qf.add_adx()
qf.add_rsi()

In [ ]:
qf.iplot()

In [ ]:
%%time
raw = {}
for s in sym:
    prices = api.get_prices_eod(s, period='d', order='a')
    price_df = pd.DataFrame.from_dict(prices).set_index('date')
    raw[s] = price_df

In [ ]:
data = pd.DataFrame()
for s in raw:
    data[s] = raw[s]['close']
data.dropna(inplace=True)

In [ ]:
data.tail()

In [ ]:
data.iplot();

In [ ]:
data.normalize(multiplier=1).iplot();

In [ ]:
data.corr()

## Bond Data

### Corporate Bonds (Not available for eod user with free plan)

For details about the corporate bond, see e.g. https://cbonds.com/bonds/470089/.
 

In [ ]:
corporate_bond = 'US00213MAS35.BOND'

In [ ]:
bond_prices = api.get_prices_eod(corporate_bond, period='w', order='a')

In [ ]:
bond_df = pd.DataFrame.from_dict(bond_prices).set_index('date')

In [ ]:
bond_df.info()

In [ ]:
bond_df.tail()

In [ ]:
bond_df['price'].iplot()

In [ ]:
# bond_df['yield'].iplot()

### Government Bond

For details, see [here](https://eodhistoricaldata.com/financial-summary/SW10Y.GBOND).

In [ ]:
government_bond = 'SW10Y.GBOND'

In [ ]:
bond_prices = api.get_prices_eod(government_bond, period='w', order='a')

In [ ]:
bond_df = pd.DataFrame.from_dict(bond_prices).set_index('date')

In [ ]:
bond_df.info()

In [ ]:
bond_df.tail()

In [ ]:
bond_df['close'].iplot()

In [ ]:
qf = cf.QuantFig(bond_df.iloc[-150:],
                title='Swiss Government 10 Year Bond',
                legend='top',
                name=government_bond)

In [ ]:
qf.iplot()

## Fundamentals Data ((Not available for eod user with free plan)

For details, see [here](https://eodhistoricaldata.com/financial-summary/600487.SHG).

In [ ]:
fundamental = api.get_fundamental_equity('600487.SHG')

In [ ]:
fundamental['General']

In [ ]:
fundamental['Highlights']

For details, see [here](https://eodhistoricaldata.com/financial-summary/ZUARI.NSE).

In [ ]:
fundamental = api.get_fundamental_equity('ZUARI.NSE')

In [ ]:
fundamental['General']

In [ ]:
fundamental['Highlights']

## Options Data (Not available for eod user with free plan)

In [ ]:
option_data = api.get_stock_options('AAPL.US')

In [ ]:
option_df = pd.DataFrame.from_dict(option_data)

In [ ]:
option_df

In [ ]:
option_df.loc[0, 'data']['expirationDate']

In [ ]:
opt_data = option_df.loc[0, 'data']

In [ ]:
opt_data.keys()

In [ ]:
for key in opt_data.keys():
    if key != 'options':
        print(key, opt_data[key])

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="30%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>